## Homework 3


> Kaggle Name: Noven_G

## import gzip
from collections import defaultdict
import numpy as np
from random import sample, randint
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import os.path
import pprint
import random
import pandas as pd

In [3]:
def readGz(f):
  for l in gzip.open(f):
    yield eval(l)

data = list(readGz('train.json.gz'))

FileNotFoundError: [Errno 2] No such file or directory: 'train.json.gz'

In [ ]:
train = data[0:10000]

val = data[100000:]

1.Although we have built a validation set, it only consists of positive samples. For this task we also need
examples of user/item pairs that weren't purchased. Build such a set by randomly sampling users and
items until you have 100,000 non-purchased user/item pairs. This random sample combined with your
100,000 validation reviews now corresponds to the complete validation set for the purchase prediction
task. Evaluate the performance (accuracy) of the baseline model on the validation set you have built

unique_reviewer1 = list(set([d['reviewerID'] for d in data]))
unique_business1 = list(set([d['itemID'] for d in data]))

In [ ]:
def parseData(fname):
    parsed_data = []
    for line in open(fname, "r"):
        parsed_data.append(line.strip())
    return parsed_data

In [ ]:
pairs_Ratings = parseData("pairs_Rating.txt")
pairs_Ratings = pairs_Ratings[1:]

In [ ]:
y_test_ratings_predictions = []
y_test_ratings_predictions_file = "predictions_Rating_test.txt"

In [ ]:
if not os.path.isfile(y_test_ratings_predictions_file):
    f = open(y_test_ratings_predictions_file, 'w')
    f.write("reviewerID-itemID,prediction\n")
    for pair in pairs_Ratings:
        u = pair.split("-")[0]
        b = pair.split("-")[1]
        uid = u[1:]
        iid = b[1:]
        pred = algo.predict(uid,iid)
        rating = pred[3]
        f.write(u + "-" + b + "," + str(rating) + "\n")
f.close()

In [ ]:
visited_pairs = []
for datum in data:
    visited_pairs.append((datum['reviewerID'], datum['itemID']))
visited_pairs = set(visited_pairs)
len(visited_pairs)

In [ ]:
non_visted_pairs = set()
counter = 100000
while counter:
    random_item_id = random.sample(unique_business1, 1)[0]
    random_userID = random.sample(unique_reviewer1, 1)[0]
    if ((random_userID, random_item_id) not in visited_pairs
        and (random_userID, random_item_id) not in non_visted_pairs):
            non_visted_pairs.add((random_userID, random_item_id))
            counter -= 1
            

In [ ]:
sampled_dataset = [[d[0], d[1]] for d in non_visted_pairs]
y_sampled = [0]*len(sampled_dataset)

In [ ]:
x_train1 = [[d['reviewerID'], d['itemID']] for d in train]
y_train1 = [1]*len(x_train)

In [ ]:
val_data1 = [[d['reviewerID'], d['itemID']] for d in val]
y_validation1 = [1]*len(val_data)

In [ ]:
val_data1 = val_data1 + sampled_dataset

x_val1 = val_data1
y_val1 = y_validation1 + y_sampled

In [ ]:
businessCount = defaultdict(int)
totalPurchases = 0
for l in x_train:
    user,business = l[0],l[1]
    businessCount[business] += 1
    totalPurchases += 1

mostPopular = [(businessCount[x], x) for x in businessCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)

    if count > totalPurchases/2:
        break

In [ ]:
y_val_predictions = []
for x in x_val:
    if x[1] in return1:
        y_val_predictions.append(1)
    else:
        y_val_predictions.append(0)


In [ ]:
print("Performance (accuracy) of the baseline model on the validation set is: ", str(accuracy_score(y_val_predictions, y_val)))

In [ ]:
predictions = open("predictions_Purchase.txt", 'w')
for l in open("pairs_Purchase.txt"):
  if l.startswith("reviewerID"):
    #header
    predictions.write(l)
    continue
  u,i = l.strip().split('-')
  if i in return1:
    predictions.write(u + '-' + i + ",1\n")
  else:
    predictions.write(u + '-' + i + ",0\n")

predictions.close()

2.The existing `purchase prediction' baseline just returns True if the item in question is `popular,' using a
threshold of the 50th percentile of popularity (totalPurchases/2). Assuming that the `non-purchased'
test examples are a random sample of user-purchase pairs, is this particular threshold value the best? If
not, see if you can nd a better one (and report its performance), or if so, explain why it is the best

The optimum threshold to get the best score in the validation set is 49% which is very close to 50% as in the baseline code. This is because  
half the reviewer/item paris is negative samples that has never happened and half is positive as in the trainig data. The predictor will perform
fairly better on the easy data than the hard ones, therefore, the best cut occurs around 50%.


In [ ]:
businessCount = defaultdict(int)
totalPurchases = 0
for l in x_train:
    user,business = l[0],l[1]
    businessCount[business] += 1
    totalPurchases += 1

mostPopular = [(businessCount[x], x) for x in businessCount]
mostPopular.sort()
mostPopular.reverse()

def partition_at(threshold):
    mostPopular_business = set()
    count = 0
    for ic, i in mostPopular:
        count += ic
        mostPopular_business.add(i)
        if count > totalPurchases*((100.0 - threshold)/100.0):
            break
    return mostPopular_business

In [ ]:
thresholds = np.arange(0, 100, 1)
threshold_accuracies = []

for t in thresholds:
    check_set = partition_at(t)
    y_validation_predictions = []
    for x in x_val:
        if x[1] in check_set:
            y_validation_predictions.append(1)
        else:
            y_validation_predictions.append(0)
    accuracy = accuracy_score(y_validation_predictions, y_val)
    threshold_accuracies.append(accuracy)

In [ ]:
for i in range(len(threshold_accuracies)):
    print("Percentile: ", str(thresholds[i]), " Accuracy: ", str(threshold_accuracies[i]))

index_max_thresh_accu = threshold_accuracies.index(max(threshold_accuracies))
print("\nA better value of accuracy is ", str(max(threshold_accuracies)))
print("It occurs at ", str(thresholds[index_max_thresh_accu]), str(" percentile."))

3.Users may tend to repeatedly purchase items of the same type. Build a baseline that returns `True' if
a user has purchased an item of the same category before (at least one category in common), or zero
otherwise

In [ ]:
## def flatten(items, seqtypes=(list, tuple)):
    ## for i, x in enumerate(items):
      ##  while i < len(items) and isinstance(items[i], seqtypes):
          ##  items[i:i+1] = items[i]
  ##  return items
## cat = [d['categories'] for d in data]

## fcat = flatten(cat[:])
## my_cat = list(set(fcat))


## reviewerIds = dict(zip(unique_reviewer, range(len(unique_reviewer))))
## businessIds = dict(zip(unique_business, range(len(unique_business))))

In [ ]:
userCategoriesVisited = defaultdict(set)
for d in train:
    uId = d['reviewerID']
    # reviewerIds[unique_reviewer]
    userCategoriesVisited[uId].add(d['categoryID'])

    
businessCategories = defaultdict(set)
for d in train:
    bId = d['itemID']
    businessCategories[bId].add(d['categoryID'])
    

In [ ]:
Popular_business = partition_at(40)

In [ ]:
def predict(uId, bId):
    if bId in Popular_business:
        cId = businessCategories[bId]
        for c in cId:
            if c in userCategoriesVisited[uId]:
                  return 1 
   

    return 0
    
  

In [ ]:
y_train_predictions = []
for x in x_train:
    uid = x[0]
    bid = x[1]
    y_train_predictions.append(predict(uid, bid))

In [ ]:
accu = accuracy_score(y_train_predictions,y_train)
print("Accuracy is ", str(accu))

In [ ]:
y_validation_predictions = []
for x in x_val:
    uid = x[0]
    bid = x[1]
    y_validation_predictions.append(predict(uid, bid))

In [ ]:
acc = accuracy_score(y_validation_predictions, y_val)
print("Accuracy is ", str(acc))

In [ ]:
predictions = open("predictions_Purchase.txt", 'w')
for l in open("pairs_Purchase.txt"):
    if l.startswith("reviewerID"):
    #header
        predictions.write(l)
        continue
    u,i = l.strip().split('-')
    predictions.write(u + "-" + i + "," + str(int(predict(u, i))) + "\n")
predictions.close()

## Part_B

Q5. What is the performance of a trivial predictor on the validation set, and what is the value of alpha?

The performance of a trival predictor on the validation set is 1.222; Alpha is 4.23

In [ ]:
def feat(d):
    return [1]

training_data = data[:100000]
x_train = [feat(d) for d in training_data]
y_train = [d['rating'] for d in training_data]

theta,residuals,rank,s = np.linalg.lstsq(x_train, y_train, rcond=None)
print("The value of alpha is ", str(theta[0]))

In [ ]:
validation_data = data[100000:]
validation_dataset = [feat(d) for d in validation_data]
y_validation = [d['rating'] for d in validation_data]
y_validation_predictions = validation_dataset*theta

sum = 0.0
for i in range(len(y_validation)):
    sum += (y_validation_predictions[i] - y_validation[i])**2
MSE_validation = sum/len(y_validation)

In [ ]:
print("MSE on validation set: ", str(MSE_validation[0]))

Q6 Report the MSE on the validation set.

MSE for validation set without negative sample is:  1.28

In [ ]:
## unique_reviewer = list(set([d['reviewerID'] for d in data]))
## unique_business = list(set([d['itemID'] for d in data]))
## reviewerIds = dict(zip(unique_reviewer, range(len(unique_reviewer))))
## businessIds = dict(zip(unique_business, range(len(unique_business))))
## num_reviewer = len(unique_reviewer)
## num_business = len(unique_business)

In [ ]:
unique_reviewer = list(set([d['reviewerID'] for d in data]))
unique_business = list(set([d['itemID'] for d in data]))
reviewerIds = dict(zip(unique_reviewer, range(len(unique_reviewer))))
businessIds = dict(zip(unique_business, range(len(unique_business))))

In [ ]:
ratings_by_users = defaultdict(dict)
ratings_for_businesses = defaultdict(dict)

for d in training_data:
    index_u = reviewerIds[d['reviewerID']]
    index_b = businessIds[d['itemID']]
    ratings_by_users[index_u][index_b] = d['rating']
    ratings_for_businesses[index_b][index_u] = d['rating']

In [ ]:
alpha = theta[0]
betaU = [1.0] * len(reviewerIds)
betaI = [1.0] * len(businessIds)

In [ ]:
def feature_for_visitedpairs(datum):
    feat = [1]
    feat.append(reviewerIds[datum['reviewerID']])
    feat.append(businessIds[datum['itemID']])
    return feat

In [ ]:
def feature_for_nonvisitedpairs(datum):
    feat = [1]
    feat.append(reviewerIds[datum.split(",")[1]])
    feat.append(businessIds[datum.split(",")[0]])
    return feat

In [ ]:

def update_alpha_beta(alpha, bU, bI, Ru, Rb, train_data, lam):
    a = 0.0
    for uid in list(range(len(bU))):
        for bid, rating in Ru[uid].items():
            a += (rating - (bU[uid] + bI[bid]))/len(train_data)
    alpha = a
    
    bU_new = [0.0]*len(bU)
    for uid in list(range(len(bU))):
        for bid, rating in Ru[uid].items():
                bU_new[uid] += (rating - (alpha + bI[bid]))/(lam + len(Ru[uid]))
    bU = bU_new
    
    bI_new = [0.0]*len(bI)
    for bid in list(range(len(bI))):
        for uid, rating in Rb[bid].items():
            bI_new[bid] += (rating - (alpha + bU[uid]))/(lam + len(Rb[bid]))
    bI = bI_new
    return a, bU_new, bI_new

In [ ]:

def find_cost(alpha, bU, bI, Ru, lam):
    bUsq = np.sum(np.square(bU))
    bIsq = np.sum(np.square(bI))
    
    term1 = 0.0
    for uid in list(range(len(bU))):
        for bid, r in Ru[uid].items():
            term1 += (alpha + bU[uid] + bI[bid] - r)**2
    return term1 + lam*(bUsq + bIsq)

In [ ]:
def RMSE_after_prediction(x, y, a, bU, bI):
    y_pred = []
    for i in range(len(x)):
        r = a*x[i][0] + bU[x[i][1]] + bI[x[i][2]]
        y_pred.append(r)
        term = 0.0
    for i in range(len(y)):
        term += (y_pred[i] - y[i])**2
    RMSE = np.sqrt(term/len(y))
    return RMSE

In [ ]:
validation_data = data[100000:]
validation_dataset = [feature_for_visitedpairs(d) for d in validation_data]
y_validation = [d['rating'] for d in validation_data]

In [ ]:
## training_data = data[:100000]
## x_train = [feature(d) for d in training_data]
## y_train = [d['rating'] for d in training_data]

In [ ]:
costs = []
RMSEs_validation = []
for i in range(100):
    alpha, betaU, betaI = update_alpha_beta(alpha, betaU, betaI, ratings_by_users, ratings_for_businesses, training_data, 1)
    costs.append(find_cost(alpha, betaU, betaI, ratings_by_users, 1))
    RMSE = RMSE_after_prediction(validation_dataset,
                                y_validation, alpha,
                                betaU, betaI)
    RMSEs_validation.append(RMSE)

In [ ]:
# len(betaU)

In [ ]:
plt.plot(list(range(100)), costs)
plt.xlabel("Number of iterations")
plt.ylabel("Cost")
plt.show()

In [ ]:
plt.plot(list(range(100)), RMSEs_validation)
plt.xlabel("Number of iterations")
plt.ylabel("RMSE on validation dataset (without non-visited pairs)")
plt.show()

In [ ]:
#print(costs[-50:])

for i in range(-100, 0):
    print(costs[i])


In [ ]:
min(costs)

> the value tends to converge at 65288.804.

In [ ]:
validation_data = data[100000:]
validation_dataset = [feature_for_visitedpairs(d) for d in validation_data]
y_validation = [d['rating'] for d in validation_data]

In [ ]:
y_validation_predictions = []
for i in range(len(validation_dataset)):
    rating = alpha*validation_dataset[i][0]
    rating += betaU[validation_dataset[i][1]]
    rating += betaI[validation_dataset[i][2]]
    y_validation_predictions.append(rating)
    
sum = 0.0
for i in range(len(y_validation)):
    sum += (y_validation_predictions[i] - y_validation[i])**2
MSE_val_set = sum/len(y_validation)


In [ ]:
print("MSE for validation set without negative sample is: ", str(MSE_val_set))

### Q7. Report the user and item IDs that have the largest and smallest values of β.

In [ ]:
x=reviewerIds.items()
xi=businessIds.items()

In [ ]:
print("User with the largest beta:", str([v[0] for i, v in enumerate(x) if v[1] == betaU.index((max(betaU)))]))

In [ ]:
print("Item with the largest beta:", str([v[0] for i, v in enumerate(xi) if v[1] == betaI.index((max(betaI)))]))

In [ ]:
print("User with the smallest beta:", str([v[0] for i, v in enumerate(x) if v[1] == betaU.index((min(betaU)))]))

In [ ]:
print("Item with the smallest beta:", str([v[0] for i, v in enumerate(xi) if v[1] == betaI.index((min(betaI)))]))

### Q8. Find a better value of λ using your validation set. Report the value you chose, its MSE, and upload your solution to Kaggle by running it on the test data.

In [ ]:
def MSE_for_prediction(x, y, a, bU, bI):
    y_pred = []
    for i in range(len(x)):
        r = a*x[i][0] + bU[x[i][1]] + bI[x[i][2]]
        y_pred.append(r)
    Sum = 0.0
    for i in range(len(y)):
        Sum += (y_pred[i] - y[i])**2
    MSE = Sum/len(y)
    return MSE

In [ ]:
lambdas = [0.1, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 20.0, 50.0, 100.0, 1000.0]

In [ ]:
validation_data = data[100000:]
validation_dataset = [feature_for_visitedpairs(d) for d in validation_data]
y_validation = [d['rating'] for d in validation_data]

In [ ]:
MSEs_validation=[]
for lam in lambdas:
    alpha = theta[0]
    betaU = [1.0] * len(reviewerIds)
    betaI = [1.0] * len(businessIds)
    for i in range(100):
        alpha, betaU, betaI = update_alpha_beta(alpha, betaU, betaI, ratings_by_users, ratings_for_businesses, training_data, lam)
    MSE = MSE_for_prediction(validation_dataset, y_validation, alpha, betaU, betaI)
    print("Lambda: ", str(lam), "MSE: ", str(MSE))
    MSEs_validation.append(MSE)

In [ ]:
print("The lambda for the smallest MSE of validation set is ", str(7.0))

In [ ]:
print("The minimum MSE of validation set is ", str(min(MSEs_validation)))

In [ ]:
### Training to optimize model

In [ ]:
training = data[:100000]
train_set = [feature_for_visitedpairs(d) for d in training]
y_training = [d['rating'] for d in training]

In [ ]:
bestlam = 7.0
alpha = theta[0]
betaU = [1.0] * len(reviewerIds)
betaI = [1.0] * len(businessIds)
    
for i in range(100):
    alpha, betaU, betaI = update_alpha_beta(alpha, betaU, betaI, ratings_by_users, ratings_for_businesses, training_data, bestlam)

In [ ]:
def pred_rating(uid, bid, a, bU, bI, ru, rb):
    rating = 0.0
    if (uid not in reviewerIds and bid in businessIds):
        for u in bU:
            rating += a + u + bI[businessIds[bid]]
        rating = rating/len(bU)
    elif (uid in reviewerIds and bid not in businessIds):
        for i in bI:
            rating += a + bU[reviewerIds[uid]] + i
        rating = rating/len(bI)
    elif (uid not in reviewerIds and bid not in businessIds):
        rating = a
    elif (uid in reviewerIds and bid in businessIds):
        rating += a + bU[reviewerIds[uid]] + bI[businessIds[bid]]
    return rating

In [ ]:
predictions = open("predictions_Rating.txt", 'w')
y_test_ratings_predictions = []
for l in open("pairs_Rating.txt"):
    if l.startswith("reviewerID"):
    #header
        predictions.write(l)
        continue
    uid, bid = l.strip().split('-')
    Rating = pred_rating(uid, bid, alpha, betaU, betaI, ratings_by_users, ratings_for_businesses)
    predictions.write(uid + '-' + bid + ',' + str(Rating) + '\n')
    y_test_ratings_predictions.append(Rating)
    
predictions.close()    